In [1]:
from importlib import import_module

In [2]:
from inspect import isfunction

In [3]:
from jobflow import job, Flow
from jobflow.managers.local import run_locally

In [4]:
def get_kwargs(lst):
    return {t['targetHandle']: {'source': t['source'], 'sourceHandle': t['sourceHandle']} for t in lst}

In [5]:
def group_edges(edges_lst):
    # edges_sorted_lst = sorted(edges_lst, key=lambda x: x['target'], reverse=True)     
    total_dict = {}
    tmp_lst = []
    target_id = edges_lst[0]['target'] 
    for ed in edges_lst:
        if target_id == ed["target"]:
            tmp_lst.append(ed)
        else:
            total_dict[target_id] = get_kwargs(lst=tmp_lst)
            target_id = ed["target"]
            tmp_lst = [ed]
    total_dict[target_id] = get_kwargs(lst=tmp_lst)
    return total_dict

In [6]:
def get_input_dict(nodes_dict):
    return {k:v for k, v in nodes_dict.items() if not isfunction(v)}

In [7]:
def get_workflow(nodes_dict, input_dict, total_dict, source_handles_dict):
    def get_attr_helper(obj, source_handle):
        print("attr_helper", source_handle, getattr(obj, "output"), getattr(getattr(obj, "output"), source_handle))
        if source_handle is None:
            return getattr(obj, "output") 
        else:
            return getattr(getattr(obj, "output"), source_handle)
    
    memory_dict = {}
    for k, v in nodes_dict.items():
        if isfunction(v):
            if k in source_handles_dict.keys():
                print(k, {el: el for el in source_handles_dict[k] if el is not None})
                fn = job(method=v, data=[el for el in source_handles_dict[k] if el is not None])
            else:
                fn = job(method=v)
            kwargs = {
                kw: input_dict[vw['source']] if vw['source'] in input_dict else get_attr_helper(obj=memory_dict[vw['source']], source_handle=vw['sourceHandle']) 
                for kw, vw in total_dict[k].items()
            }
            print(k, kwargs)
            memory_dict[k] = fn(**kwargs)
    return list(memory_dict.values())

In [8]:
def get_source_handles(edges_lst):
    source_handle_dict = {}
    for ed in edges_lst: 
        if ed['source'] not in source_handle_dict.keys():
            source_handle_dict[ed['source']] = [ed['sourceHandle']]
        else:
            source_handle_dict[ed['source']].append(ed['sourceHandle'])
    return source_handle_dict

In [9]:
def get_item_from_tuple(input_obj, index, index_lst):
    if isinstance(input_obj, dict):  
        return input_obj[index]
    else:  # input_obj is a tuple
        return list(input_obj)[index_lst.index(index)]

In [10]:
def get_source_handles(edges_lst):
    source_handle_dict = {}
    for ed in edges_lst: 
        if ed['source'] not in source_handle_dict.keys():
            source_handle_dict[ed['source']] = [ed['sourceHandle']]
        else:
            source_handle_dict[ed['source']].append(ed['sourceHandle'])
    return source_handle_dict

In [11]:
def add_xy(x, y):
    return {"x":x, "y": y, "z": x + y}

In [12]:
def add_xyz(x, y, z):
    print(x, y, z)
    return x + y + z

In [13]:
nodes_dict = {
    0: add_xy,
    1: add_xyz,
    2: 1,
    3: 2,
}

In [14]:
edges_lst = [
    {'target': 1, 'targetHandle': 'x', 'source': 0, 'sourceHandle': 'x'},
    {'target': 1, 'targetHandle': 'y', 'source': 0, 'sourceHandle': 'y'},
    {'target': 1, 'targetHandle': 'z', 'source': 0, 'sourceHandle': 'z'},
    {'target': 0, 'targetHandle': 'x', 'source': 2, 'sourceHandle': None},
    {'target': 0, 'targetHandle': 'y', 'source': 3, 'sourceHandle': None},
]

In [15]:
source_handles_dict = get_source_handles(edges_lst=edges_lst)
source_handles_dict

{0: ['x', 'y', 'z'], 2: [None], 3: [None]}

In [16]:
total_dict = group_edges(edges_lst=edges_lst)
input_dict = get_input_dict(nodes_dict=nodes_dict)

In [17]:
total_dict, input_dict

({1: {'x': {'source': 0, 'sourceHandle': 'x'},
   'y': {'source': 0, 'sourceHandle': 'y'},
   'z': {'source': 0, 'sourceHandle': 'z'}},
  0: {'x': {'source': 2, 'sourceHandle': None},
   'y': {'source': 3, 'sourceHandle': None}}},
 {2: 1, 3: 2})

In [18]:
task_lst = get_workflow(
    nodes_dict=nodes_dict, 
    input_dict=input_dict, 
    total_dict=total_dict,
    source_handles_dict=source_handles_dict,
)

0 {'x': 'x', 'y': 'y', 'z': 'z'}
0 {'x': 1, 'y': 2}
attr_helper x OutputReference(23a8b6db-98ac-476a-b865-70bff6979812) OutputReference(23a8b6db-98ac-476a-b865-70bff6979812, .x)
attr_helper y OutputReference(23a8b6db-98ac-476a-b865-70bff6979812) OutputReference(23a8b6db-98ac-476a-b865-70bff6979812, .y)
attr_helper z OutputReference(23a8b6db-98ac-476a-b865-70bff6979812) OutputReference(23a8b6db-98ac-476a-b865-70bff6979812, .z)
1 {'x': OutputReference(23a8b6db-98ac-476a-b865-70bff6979812, .x), 'y': OutputReference(23a8b6db-98ac-476a-b865-70bff6979812, .y), 'z': OutputReference(23a8b6db-98ac-476a-b865-70bff6979812, .z)}


In [19]:
flow = Flow(task_lst)

In [20]:
result = run_locally(flow)
result

2025-01-17 07:39:02,400 INFO Started executing jobs locally
2025-01-17 07:39:04,449 INFO Starting job - add_xy (23a8b6db-98ac-476a-b865-70bff6979812)
2025-01-17 07:39:04,450 INFO Finished job - add_xy (23a8b6db-98ac-476a-b865-70bff6979812)
2025-01-17 07:39:04,451 INFO Starting job - add_xyz (85ace7ba-83dd-4ab2-b261-b6b2b2f8dc70)
1 2 3
2025-01-17 07:39:04,452 INFO Finished job - add_xyz (85ace7ba-83dd-4ab2-b261-b6b2b2f8dc70)
2025-01-17 07:39:04,453 INFO Finished executing jobs locally


{'23a8b6db-98ac-476a-b865-70bff6979812': {1: Response(output={'x': 1, 'y': 2, 'z': 3}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/janssen/notebooks/2025/2025-01-16-jobflow-qe'))},
 '85ace7ba-83dd-4ab2-b261-b6b2b2f8dc70': {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/janssen/notebooks/2025/2025-01-16-jobflow-qe'))}}